In [1]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
df = pd.read_csv('RFM_ht_data.csv')

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df

,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00
...,...,...,...,...
332725,S0081810310459,14092500,2020-09-30,3801.87
332726,S0081810310461,99065678,2020-09-30,5769.88
332727,S0081810310462,19029918,2020-09-30,736.88
332728,S0081810310463,13020033,2020-09-30,1475.20


In [4]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate, format='%Y-%m-%d %H:%M:%S')
df.InvoiceNo = df.InvoiceNo.astype(str)
df.CustomerCode = df.CustomerCode.astype(str)

In [5]:
NOW = df.InvoiceDate.max()
NOW

Timestamp('2020-09-30 00:00:00')

In [6]:
#Создаем датафрейм с количеством прошедших дней от последней покупи в нашем датафрейме
#с количеством покупок для каждого пользователя и с потраченной суммой на наши покупки
RFM_table= df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                                'InvoiceNo': lambda x: len(x.unique()), # Frequency
                                                'Amount': lambda x: x.sum()})    # Monetary 

In [7]:
#Переименовываем колонки для удобства
RFM_table['InvoiceDate'] = RFM_table['InvoiceDate'].astype(int)
RFM_table.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency',
                         'Amount': 'monetary_value'}, inplace=True)

In [8]:
RFM_table = RFM_table.reset_index()

In [9]:
RFM_table

,CustomerCode,recency,frequency,monetary_value
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [10]:
quantiles = RFM_table[['recency', 'frequency', 'monetary_value']].quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [11]:
#Конвертируем quantiles в словарь для более простого использования
quantiles = quantiles.to_dict()

In [12]:
#Создаем датафрейм с RFM сегментацией
RFM_Segment = RFM_table.copy()

In [13]:
#Функция для кластеризации квантилей: для меньших квантилей меньший кластер
def FM_Class(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [14]:
#Функция для кластеризации квантилей: для меньших квантилей больший кластер
def R_Class(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [15]:
RFM_table

,CustomerCode,recency,frequency,monetary_value
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88
...,...,...,...,...
123728,99099927,10,1,961.10
123729,99099936,0,1,1521.78
123730,99099959,8,2,1444.56
123731,99099963,19,1,3018.91


In [16]:
#Создаем колонки с нашими кластерами и дабавляем в датафрейм с RFM сегментацией
RFM_Segment['R_Quartile'] = RFM_Segment['recency'].apply(FM_Class, args=('recency',quantiles,))
RFM_Segment['F_Quartile'] = RFM_Segment['frequency'].apply(R_Class, args=('frequency',quantiles,))
RFM_Segment['M_Quartile'] = RFM_Segment['monetary_value'].apply(R_Class, args=('monetary_value',quantiles,))
RFM_Segment['RFMClass'] = RFM_Segment.R_Quartile.map(str) \
                            + RFM_Segment.F_Quartile.map(str) \
                            + RFM_Segment.M_Quartile.map(str)

In [17]:
RFM_Segment

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443
...,...,...,...,...,...,...,...,...
123728,99099927,10,1,961.10,3,4,3,343
123729,99099936,0,1,1521.78,1,4,3,143
123730,99099959,8,2,1444.56,2,3,3,233
123731,99099963,19,1,3018.91,4,4,2,442


In [18]:
RFM_Segment.sort_values('frequency', ascending=False)

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
89388,19057820,0,204,104871.80,1,1,1,111
44594,13215452,0,113,85334.40,1,1,1,111
10347,13032521,0,106,389309.92,1,1,1,111
97077,19080880,27,99,169930.72,4,1,1,411
119951,99003061,0,90,398759.35,1,1,1,111
...,...,...,...,...,...,...,...,...
58910,13272861,0,1,3312.62,1,4,2,142
58911,13272871,12,1,684.80,3,4,4,344
58913,13272875,4,1,4284.89,2,4,1,241
58914,13272878,5,1,366.01,2,4,4,244


In [19]:
RFM_Segment.query('RFMClass == "311"')

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
428,10001834,13,6,5855.21,3,1,1,311
430,10001842,9,5,7219.98,3,1,1,311
551,10003108,9,5,6831.49,3,1,1,311
576,10003319,11,4,7588.12,3,1,1,311
696,10004583,9,11,13527.71,3,1,1,311
...,...,...,...,...,...,...,...,...
123243,99089535,9,4,6394.49,3,1,1,311
123361,99092052,11,4,6237.87,3,1,1,311
123433,99093473,15,7,8483.54,3,1,1,311
123620,99097473,12,6,6972.62,3,1,1,311


In [20]:
RFM_Segment.groupby('RFMClass', as_index=False).agg({'CustomerCode': 'count'}).sort_values('CustomerCode', ascending=False)

,RFMClass,CustomerCode
63,444,10624
0,111,9705
62,443,6729
47,344,6593
16,211,5847
...,...,...
55,424,63
19,214,60
3,114,60
35,314,33


# Задание 1
    Какое максимальное кол-во покупок было совершено одним пользователем?

In [23]:
RFM_table.frequency.max()

204

# Задание 2
    Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [44]:
quantiles['monetary_value'][0.25]

765.0

# Задание 3
    Какая верхняя граница у суммы покупок у пользователей с классом 4 в подсегменте М? (Другими словами: пользователи, у которых сумма покупок от 0 до Х попадают в 4 класс в подсегменте М)

In [46]:
quantiles['frequency'][0.75]

3.0

# Задание 4
    Какое максимальное количество дней может пройти с момента последней покупки для того, чтобы пользователь попал в класс 2 в подсегменте R?

In [48]:
quantiles['recency'][0.5]

8.0

# Задание 5 
    Сколько пользователей попало в сегмент 111?

In [55]:
RFM_Segment.query('RFMClass == "111"').CustomerCode.count()

9705

# Задание 5
    Сколько пользователей попало в сегмент 311?

In [56]:
RFM_Segment.query('RFMClass == "311"').CustomerCode.count()

1609

# Задание 6
    В каком RFM-сегменте самое большое кол-во пользователей?

In [71]:
RFM_Segment.groupby('RFMClass', as_index=False).agg({'CustomerCode': 'count'}) \
    .sort_values('CustomerCode', ascending=False).reset_index(drop=True).RFMClass[0]

'444'

# Задание 7
    В каком RFM-сегменте самое маленькое кол-во пользователей?

In [98]:
RFM_Segment.groupby('RFMClass', as_index=False).agg({'CustomerCode': 'count'}) \
    .sort_values('CustomerCode', ascending=False).reset_index(drop=True).RFMClass

0     444
1     111
2     443
3     344
4     211
     ... 
59    424
60    214
61    114
62    314
63    414
Name: RFMClass, Length: 64, dtype: object

In [107]:
RFM_Segment.groupby('RFMClass', as_index=False).agg({'CustomerCode': 'count'}) \
    .sort_values('CustomerCode').reset_index(drop=True).RFMClass[0]

'414'

# Задание 8
    Какое количество пользователей попало в самый малочисленный сегмент?

In [112]:
RFM_Segment.groupby('RFMClass', as_index=False).agg({'CustomerCode': 'count'}) \
    .sort_values('CustomerCode').reset_index(drop=True).CustomerCode[0]

2